In [1]:
import argparse
import sys
import os
import time
import datetime
import ast
from collections import Counter
from pathlib import Path
import hydra

import numpy as np
import torch

# Ensure local src/ is importable
sys.path.insert(0, str(Path(os.getcwd()).parent / "src"))

from my_genQC.inference.eval_metrics import UnitaryFrobeniusNorm, UnitaryInfidelityNorm
from my_genQC.inference.evaluation_helper import get_unitaries, get_srvs
from my_genQC.inference.sampling import generate_compilation_tensors, generate_tensors, decode_tensors_to_backend
from my_genQC.pipeline.diffusion_pipeline import DiffusionPipeline
from my_genQC.platform.simulation import Simulator, CircuitBackendType
from my_genQC.platform.tokenizer.circuits_tokenizer import CircuitTokenizer
from my_genQC.utils.misc_utils import infer_torch_device, get_entanglement_bins
from my_genQC.dataset import circuits_dataset
from my_genQC.models.config_model import ConfigModel
from my_genQC.utils.config_loader import load_config, store_tensor, load_tensor

from quantum_diffusion.data.dataset import DatasetLoader
from quantum_diffusion.evaluation.evaluator import SRVEvaluator

In [2]:
# Config loading
with hydra.initialize(version_base=None, config_path="../conf"):
    cfg = hydra.compose(config_name="config.yaml", overrides=["evaluation=remote_model"])
    cfg = cfg["evaluation"]

In [3]:
def evaluate_remote_model(num_qubits: int, tensor_out_file, config):
    config.dataset = f"../datasets/paper_quditkit/srv_{num_qubits}q_dataset"
    config.wandb.run_name = f"{num_qubits}q_remote_model_evaluation"
    evaluator = SRVEvaluator(config=cfg)

    tensors_out = load_tensor(f"../scripts/inference/{tensor_out_file}", device="cuda")

    decoded_circuits = evaluator.decode_tensors(tensors_out)
    valid_indices, target_srvs, predicted_srvs = evaluator.validate_and_calculate_srvs(decoded_circuits)
    srv_exact_match_rate, acc_per_entanglement = evaluator.calculate_metrics(target_srvs, predicted_srvs)

    return evaluator, decoded_circuits, valid_indices, target_srvs, predicted_srvs, srv_exact_match_rate, acc_per_entanglement

## 3q Evaluation

In [4]:
q3_evaluator, q3_decoded_circuits, q3_valid_indices, q3_target_srvs, q3_predicted_srvs, q3_srv_exact_match_rate, q3_acc_per_entanglement = evaluate_remote_model(3, "3q_163327_samples.pt", cfg)

[INFO]: Cuda device has a capability of 8.6 (>= 8), allowing tf32 matmul.
2026-01-02 22:46:23 - quantum_diffusion.evaluation.evaluator - INFO - Setting up wandb...


wandb: Currently logged in as: liondungl (lions-team) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


[INFO]: Loading tensor from `../datasets/paper_quditkit/srv_3q_dataset/dataset/ds_x.safetensors` onto device: cuda.
[INFO]: Loading tensor from `../datasets/paper_quditkit/srv_3q_dataset/dataset/ds_y.safetensors` onto device: cuda.
[INFO]: Instantiated config_dataset from given config on cuda.
2026-01-02 22:46:25 - quantum_diffusion.data.dataset - INFO - Dataset loaded from ../datasets/paper_quditkit/srv_3q_dataset


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

[INFO]: `genQC.models.unet_qc.QC_Cond_UNet` instantiated from given `config` on cuda.
[INFO]: Loading model from `/root/.cache/huggingface/hub/models--Floki00--qc_srv_3to8qubit/snapshots/18eece95eb50483c46dd4eef1cdac3e09afadd54/model.pt` onto device: cuda.
[INFO]: `genQC.models.unet_qc.QC_Cond_UNet`. Freeze model: True
[INFO]: `genQC.models.frozen_open_clip.CachedFrozenOpenCLIPEmbedder` instantiated from given `config` on cuda.
[INFO]: `genQC.models.frozen_open_clip.CachedFrozenOpenCLIPEmbedder`. Found no key `save_type` in `config`. No state dict loaded.
[INFO]: `genQC.models.frozen_open_clip.CachedFrozenOpenCLIPEmbedder`. Freeze model: True
[WARNING]: The value 0 is reserved for background tokens, i.e. qubit time position which are not effected by gates.
[WARNING]: Automatically incrementing all vocabulary values by one ...
[INFO]: Loading tensor from `../scripts/inference/3q_163327_samples.pt` onto device: cuda.
2026-01-02 22:46:29 - quantum_diffusion.evaluation.evaluator - INFO - D

## 4q Evaluation

In [5]:
q4_evaluator, q4_decoded_circuits, q4_valid_indices, q4_target_srvs, q4_predicted_srvs, q4_srv_exact_match_rate, q4_acc_per_entanglement = evaluate_remote_model(4, "4q_281970_samples.pt", cfg)

[INFO]: Cuda device has a capability of 8.6 (>= 8), allowing tf32 matmul.
2026-01-02 22:50:52 - quantum_diffusion.evaluation.evaluator - INFO - Setting up wandb...


eval/decode_failures,719
eval/decoded_circuits,162608
eval/n_entangled_acc/0,0.99507
eval/n_entangled_acc/2,0.78928
eval/n_entangled_acc/3,0.52352
eval/srv_exact_match_rate,0.84964


[INFO]: Loading tensor from `../datasets/paper_quditkit/srv_4q_dataset/dataset/ds_x.safetensors` onto device: cuda.
[INFO]: Loading tensor from `../datasets/paper_quditkit/srv_4q_dataset/dataset/ds_y.safetensors` onto device: cuda.
[INFO]: Instantiated config_dataset from given config on cuda.
2026-01-02 22:50:55 - quantum_diffusion.data.dataset - INFO - Dataset loaded from ../datasets/paper_quditkit/srv_4q_dataset


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

[INFO]: `genQC.models.unet_qc.QC_Cond_UNet` instantiated from given `config` on cuda.
[INFO]: Loading model from `/root/.cache/huggingface/hub/models--Floki00--qc_srv_3to8qubit/snapshots/18eece95eb50483c46dd4eef1cdac3e09afadd54/model.pt` onto device: cuda.
[INFO]: `genQC.models.unet_qc.QC_Cond_UNet`. Freeze model: True
[INFO]: `genQC.models.frozen_open_clip.CachedFrozenOpenCLIPEmbedder` instantiated from given `config` on cuda.
[INFO]: `genQC.models.frozen_open_clip.CachedFrozenOpenCLIPEmbedder`. Found no key `save_type` in `config`. No state dict loaded.
[INFO]: `genQC.models.frozen_open_clip.CachedFrozenOpenCLIPEmbedder`. Freeze model: True
[WARNING]: The value 0 is reserved for background tokens, i.e. qubit time position which are not effected by gates.
[WARNING]: Automatically incrementing all vocabulary values by one ...
[INFO]: Loading tensor from `../scripts/inference/4q_281970_samples.pt` onto device: cuda.
2026-01-02 22:50:58 - quantum_diffusion.evaluation.evaluator - INFO - D

In [6]:
q5_evaluator, q5_decoded_circuits, q5_valid_indices, q5_target_srvs, q5_predicted_srvs, q5_srv_exact_match_rate, q5_acc_per_entanglement = evaluate_remote_model(5, "5q_497496_samples.pt", cfg)

[INFO]: Cuda device has a capability of 8.6 (>= 8), allowing tf32 matmul.
2026-01-02 22:58:12 - quantum_diffusion.evaluation.evaluator - INFO - Setting up wandb...


eval/decode_failures,1896
eval/decoded_circuits,280074
eval/n_entangled_acc/0,0.99958
eval/n_entangled_acc/2,0.92067
eval/n_entangled_acc/3,0.74466
eval/n_entangled_acc/4,0.64401
eval/srv_exact_match_rate,0.8898


[INFO]: Loading tensor from `../datasets/paper_quditkit/srv_5q_dataset/dataset/ds_x.safetensors` onto device: cuda.
[INFO]: Loading tensor from `../datasets/paper_quditkit/srv_5q_dataset/dataset/ds_y.safetensors` onto device: cuda.
[INFO]: Instantiated config_dataset from given config on cuda.
2026-01-02 22:58:15 - quantum_diffusion.data.dataset - INFO - Dataset loaded from ../datasets/paper_quditkit/srv_5q_dataset


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

[INFO]: `genQC.models.unet_qc.QC_Cond_UNet` instantiated from given `config` on cuda.
[INFO]: Loading model from `/root/.cache/huggingface/hub/models--Floki00--qc_srv_3to8qubit/snapshots/18eece95eb50483c46dd4eef1cdac3e09afadd54/model.pt` onto device: cuda.
[INFO]: `genQC.models.unet_qc.QC_Cond_UNet`. Freeze model: True
[INFO]: `genQC.models.frozen_open_clip.CachedFrozenOpenCLIPEmbedder` instantiated from given `config` on cuda.
[INFO]: `genQC.models.frozen_open_clip.CachedFrozenOpenCLIPEmbedder`. Found no key `save_type` in `config`. No state dict loaded.
[INFO]: `genQC.models.frozen_open_clip.CachedFrozenOpenCLIPEmbedder`. Freeze model: True
[WARNING]: The value 0 is reserved for background tokens, i.e. qubit time position which are not effected by gates.
[WARNING]: Automatically incrementing all vocabulary values by one ...
[INFO]: Loading tensor from `../scripts/inference/5q_497496_samples.pt` onto device: cuda.
2026-01-02 22:58:19 - quantum_diffusion.evaluation.evaluator - INFO - D

KeyboardInterrupt: 

In [ ]:
q6_evaluator, q6_decoded_circuits, q6_valid_indices, q6_target_srvs, q6_predicted_srvs, q6_srv_exact_match_rate, q6_acc_per_entanglement = evaluate_remote_model(6, "6q_499926_samples.pt", cfg)

In [ ]:
q7_evaluator, q7_decoded_circuits, q7_valid_indices, q7_target_srvs, q7_predicted_srvs, q7_srv_exact_match_rate, q7_acc_per_entanglement = evaluate_remote_model(7, "7q_499968_samples.pt", cfg)

In [ ]:
q8_evaluator, q8_decoded_circuits, q8_valid_indices, q8_target_srvs, q8_predicted_srvs, q8_srv_exact_match_rate, q8_acc_per_entanglement = evaluate_remote_model(8, "8q_599936_samples.pt", cfg)